# Linear SVM & Poly SVM Using Variable Selection

In [88]:
import pandas as pd
import numpy as np
from numpy import mean
from numpy import absolute
from matplotlib import pyplot as plt
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from scipy.stats import randint
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import warnings
warnings.filterwarnings('ignore')

data=pd.read_csv("Group_2_data_cleaned.csv")
data=pd.DataFrame(data)
X=np.array(data.iloc[0:,3:12])
Y=np.array(data.iloc[0:,12:])

In [89]:
LE=LabelEncoder()
X[:,0]=LE.fit_transform(X[:,0]) #Select labels from variables
#Split train and test data. Test data size is 50%, training set is 50%
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.5, random_state = 42) 
#The data set is normalized or 'standardized' for a better performaing KNN for both train and test data
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
#define K-fold cross-validation method to use
cv = KFold(n_splits=10, random_state=1, shuffle=True)

## Without Feature Selection

### Part-2

#### Linear Classifier(SGD)

In [90]:
sgd = make_pipeline(StandardScaler(),
                    SGDClassifier(max_iter=1000, tol=1e-3, loss="perceptron", penalty="l1"))
start_time = time.time()
sgd.fit(X_train, y_train) 
end_time = time.time()
dur1=end_time - start_time 

In [91]:
sgd_pred=sgd.predict(X_test)
sgd_pred

array([0, 0, 1, ..., 1, 0, 0])

In [92]:
print(classification_report(y_test, sgd_pred))

              precision    recall  f1-score   support

           0       0.99      0.98      0.98      2821
           1       0.85      0.91      0.88       340

    accuracy                           0.97      3161
   macro avg       0.92      0.94      0.93      3161
weighted avg       0.97      0.97      0.97      3161



In [93]:
acc1=accuracy_score(y_test, sgd_pred)*100
print("Accuracy Percent:", round(acc1,2))

Accuracy Percent: 97.25


#### KNN

In [94]:
knn = KNeighborsClassifier(n_neighbors = 3, metric = 'minkowski', p = 2) #p=2 means euclidian distance
start_time = time.time()
knn.fit(X_train, y_train)
end_time = time.time()
dur2=end_time - start_time
# Use K-Fold Cross Validation
scores = cross_val_score(knn, X, Y, scoring='neg_mean_squared_error',
                         cv=cv, n_jobs=-1)

#Normal 50/50 Cross Validation
knn_pred = knn.predict(X_test)
from sklearn.metrics import confusion_matrix,accuracy_score
ConfMat = confusion_matrix(y_test, knn_pred) #Confusion Matrix
AccScore = accuracy_score(y_test,knn_pred) #Accuracy
#Given function for accuracy

/Users/akashborigi/anaconda3/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/akashborigi/anaconda3/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/akashborigi/anaconda3/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/akashborigi/anaconda3/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1

In [95]:
print(classification_report(y_test, knn_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      2821
           1       0.92      0.95      0.93       340

    accuracy                           0.99      3161
   macro avg       0.96      0.97      0.96      3161
weighted avg       0.99      0.99      0.99      3161



In [96]:
acc2=round(AccScore,4)*100
print('Accuracy:', round(AccScore,4)*100,'%')

Accuracy: 98.54 %


#### Random Forest

In [97]:
#random_search cv used for the hyper-parameter tuning
param_dist = {'n_estimators': randint(50, 200),
              'max_depth': randint(3, 10),
              'min_samples_split': randint(2, 20),
              'min_samples_leaf': randint(1, 10),
             'random_state': randint(10,42,78)}
r = RandomForestClassifier()
# perform the random search
rf = RandomizedSearchCV(estimator=r, param_distributions=param_dist, n_iter=10, cv=cv, random_state=42)
start_time = time.time()
rf.fit(X_train, y_train)
end_time = time.time()
dur3=end_time - start_time
rf_pred = rf.predict(X_test)

In [98]:
print(classification_report(y_test, rf_pred))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      2821
           1       0.90      0.98      0.94       340

    accuracy                           0.99      3161
   macro avg       0.95      0.98      0.96      3161
weighted avg       0.99      0.99      0.99      3161



In [99]:
print("Best Hyper-parameters", rf.best_params_)

Best Hyper-parameters {'max_depth': 8, 'min_samples_leaf': 6, 'min_samples_split': 3, 'n_estimators': 70, 'random_state': 88}


In [100]:
acc3=accuracy_score(y_test, rf_pred)*100
print("Accuracy Percent:", round(acc3,2))

Accuracy Percent: 98.54


#### SVM- Linear

In [101]:
#Create a svm Classifier
lin_svc = svm.SVC(kernel='linear') # Linear Kernel
 
scores = cross_val_score(lin_svc, X, Y, scoring='neg_mean_squared_error',
                         cv=cv, n_jobs=-1)

#Train the model using the training sets
start_time = time.time()
lin_svc.fit(X_train, y_train)
end_time = time.time()
dur4=end_time - start_time

#Predict the response for test dataset
svc_pred = lin_svc.predict(X_test)

/Users/akashborigi/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/akashborigi/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/akashborigi/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/akashborigi/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d arr

In [102]:
acc4=accuracy_score(y_test, svc_pred)*100
print("Accuracy:",accuracy_score(y_test, svc_pred)*100,'%')
print('Confusion Matrix:\n',confusion_matrix(y_test, svc_pred))

Accuracy: 97.69060423916483 %
Confusion Matrix:
 [[2753   68]
 [   5  335]]


In [103]:
print(classification_report(y_test, svc_pred))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99      2821
           1       0.83      0.99      0.90       340

    accuracy                           0.98      3161
   macro avg       0.91      0.98      0.94      3161
weighted avg       0.98      0.98      0.98      3161



## Part-3

#### Non-Linear SVM Classifier

In [104]:
# Set up the pipeline
svm_pipe = make_pipeline(StandardScaler(), SVC())

# Set up the grid search
param_grid = {
    'svc__C': [0.1, 1, 10],
    'svc__kernel': ['poly', 'rbf', 'sigmoid'],
    'svc__gamma': ['scale', 'auto']
}
clf = GridSearchCV(svm_pipe, param_grid, cv=5)

# Fit the model and time it
clf.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('svc', SVC())]),
             param_grid={'svc__C': [0.1, 1, 10],
                         'svc__gamma': ['scale', 'auto'],
                         'svc__kernel': ['poly', 'rbf', 'sigmoid']})

In [105]:
# Print the best parameters and the time taken to fit the model
print('Best parameters:', clf.best_params_)

Best parameters: {'svc__C': 10, 'svc__gamma': 'scale', 'svc__kernel': 'rbf'}


In [106]:
# Use the best parameters to fit a new SVM model
best_params = clf.best_params_
svm_pipe = make_pipeline(StandardScaler(), SVC(C=best_params['svc__C'], kernel=best_params['svc__kernel'], gamma=best_params['svc__gamma']))
start_time = time.time()
svm_pipe.fit(X_train, y_train)
end_time = time.time()
dur6=end_time - start_time

In [107]:
y_pred = svm_pipe.predict(X_test)
print(classification_report(y_test, y_pred))
print('Confusion Matrix:\n', confusion_matrix(y_pred,y_test),'\n')
y_pred_nl = svm_pipe.predict(X_test)
print(classification_report(y_test, y_pred_nl))
acc_nl=accuracy_score(y_test, y_pred_nl)*100
print("Accuracy Percent:", round(acc_nl,2))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      2821
           1       0.95      0.98      0.96       340

    accuracy                           0.99      3161
   macro avg       0.97      0.99      0.98      3161
weighted avg       0.99      0.99      0.99      3161

Confusion Matrix:
 [[2803    8]
 [  18  332]] 

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      2821
           1       0.95      0.98      0.96       340

    accuracy                           0.99      3161
   macro avg       0.97      0.99      0.98      3161
weighted avg       0.99      0.99      0.99      3161

Accuracy Percent: 99.18


#### ELM

In [108]:
class ELM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ELM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        h = F.relu(self.linear1(x))
        y_pred = torch.sigmoid(self.linear2(h))
        return y_pred

In [109]:
# Perform model structure selection
parameters = {'hidden_size': [64, 128], 'batch_size': [32, 64], 'learning_rate': [0.001, 0.01]}

In [110]:
start_time = time.time()
for hidden_size in parameters['hidden_size']:
    for batch_size in parameters['batch_size']:
        for learning_rate in parameters['learning_rate']:
            elm = ELM(X_train.shape[1], hidden_size, 1)
            optimizer = torch.optim.Adam(elm.parameters(), lr=learning_rate)
            criterion = nn.BCELoss()

            # Train the ELM
            for i in range(0, len(X_train), batch_size):
                x = torch.Tensor(X_train[i:i+batch_size])
                y = torch.Tensor(y_train[i:i+batch_size])
                y_pred = elm(x)
                loss = criterion(y_pred, y)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            # Test the ELM
            y_pred = elm(torch.Tensor(X_test)).detach().numpy()
            y_pred = [1 if p > 0.5 else 0 for p in y_pred]
end_time = time.time()
dur7=end_time - start_time

In [111]:
print('hidden_size: {}, batch_size: {}, learning_rate: {}'.format(hidden_size, batch_size, learning_rate))
print(classification_report(y_test, y_pred))
print('Confusion Matrix:\n', confusion_matrix(y_pred,y_test),'\n')
acc_elm=accuracy_score(y_test, y_pred)*100
print("Accuracy Percent:\n", round(acc_elm,2))

hidden_size: 128, batch_size: 64, learning_rate: 0.01
              precision    recall  f1-score   support

           0       0.99      0.98      0.98      2821
           1       0.84      0.90      0.87       340

    accuracy                           0.97      3161
   macro avg       0.91      0.94      0.93      3161
weighted avg       0.97      0.97      0.97      3161

Confusion Matrix:
 [[2762   34]
 [  59  306]] 

Accuracy Percent:
 97.06


#### Deep Learning 

In [112]:
#Create and initialize deep learning classifier
gaussDL = GaussianNB()
DLmodel = gaussDL.fit(X_train, y_train)
start_time = time.time()
y_pred3 = gaussDL.predict(X_test)
end_time = time.time()
dur8=end_time - start_time

In [113]:
print(classification_report(y_test, y_pred3))
print('Confusion Matrix:\n',confusion_matrix(y_test, y_pred3))
acc_dl=accuracy_score(y_test, y_pred3)*100
print("Accuracy Percent:\n", round(acc_dl,2))

              precision    recall  f1-score   support

           0       1.00      0.97      0.98      2821
           1       0.79      0.99      0.88       340

    accuracy                           0.97      3161
   macro avg       0.90      0.98      0.93      3161
weighted avg       0.98      0.97      0.97      3161

Confusion Matrix:
 [[2734   87]
 [   3  337]]
Accuracy Percent:
 97.15


### Comparative Results:

In [114]:
print("SGD",acc1)
print("KNN",acc2)
print("Random Forest",acc3)
print("SVM_Linear",acc4)
print("SVM_Non Linear",acc_nl)
print("ELM",acc_elm)
print("Deep Learning",acc_dl)

SGD 97.24770642201835
KNN 98.54
Random Forest 98.54476431509016
SVM_Linear 97.69060423916483
SVM_Non Linear 99.17747548244226
ELM 97.05789307181271
Deep Learning 97.15279974691553


In [115]:
print(f"SGD Model Fitting Time: {dur1}")
print(f"KNN Model Fitting Time: {dur2}")
print(f"Random Forest Model Fitting Time: {dur3}")
print(f"SVM_Linear Model Fitting Time: {dur4}")
print(f"SVM_Non Linear Model Fitting Time: {dur6}")
print(f"ELM Model Fitting Time: {dur7}")
print(f"Deep Learning Model Fitting Time: {dur8}")

SGD Model Fitting Time: 0.017282724380493164
KNN Model Fitting Time: 0.002972126007080078
Random Forest Model Fitting Time: 16.62758183479309
SVM_Linear Model Fitting Time: 0.01803278923034668
SVM_Non Linear Model Fitting Time: 0.02216315269470215
ELM Model Fitting Time: 0.17454266548156738
Deep Learning Model Fitting Time: 0.00041604042053222656


## With Feature Selection

### Correlation Variable Selection

##### Bidding_Ratio, Successive_Outbidding, Last_Bidding, Auction_Bids, Starting_Price_Average, Early_Bidding, Winning Ratio

In [11]:
#Variable Selection
X_train_VS=X_train[0:,1:7]
y_train_VS=y_train
X_test_VS=X_test[0:,1:7]
y_test_VS=y_test

#### Best Model Part 2 (SVM LINEAR)

In [87]:
# Based on accuracy performance and best time complexity, we selected SVM linear as best model
model = LinearSVC(penalty='l2', dual=False, max_iter=10000)
param_grid = {'C': [0.01, 0.1, 1, 10, 100]}

In [13]:
start_time = time.time()
# Set up k-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
# Set up GridSearchCV to perform model selection using k-fold cross-validation
clf = GridSearchCV(model, param_grid=param_grid, cv=kfold)
clf.fit(X_train, y_train)
end_time = time.time()
duration1=end_time-start_time
# Get the best model
best_model = clf.best_estimator_
print(f"Best model: {best_model}")

Best model: LinearSVC(C=0.01, dual=False, max_iter=10000)


In [14]:
# Evaluate the best model on the testing data
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(classification_report(y_test, y_pred))
print('Confusion Matrix:\n', confusion_matrix(y_pred,y_test),'\n')
print(f"Accuracy: {round(accuracy*100,2)}%")

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      2821
           1       0.83      0.95      0.89       340

    accuracy                           0.97      3161
   macro avg       0.91      0.96      0.94      3161
weighted avg       0.98      0.97      0.98      3161

Confusion Matrix:
 [[2757   17]
 [  64  323]] 

Accuracy: 97.44%


#### Best Model Part 3 (SVM Non-Linear)

In [15]:
# SVM Nonlinear

# Set up the pipeline
svm_pipe = make_pipeline(StandardScaler(), SVC())

# Set up the grid search
param_grid = {
    'svc__C': [0.1, 1, 10],
    'svc__kernel': ['poly', 'rbf', 'sigmoid'],
    'svc__gamma': ['scale', 'auto']
}
clf = GridSearchCV(svm_pipe, param_grid, cv=5)
# Fit the model and time it
clf.fit(X_train_VS, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('svc', SVC())]),
             param_grid={'svc__C': [0.1, 1, 10],
                         'svc__gamma': ['scale', 'auto'],
                         'svc__kernel': ['poly', 'rbf', 'sigmoid']})

In [16]:
#Print the best parameters and the time taken to fit the model
print('Best parameters:', clf.best_params_)

Best parameters: {'svc__C': 10, 'svc__gamma': 'scale', 'svc__kernel': 'rbf'}


In [17]:
# Use the best parameters to fit a new SVM model
best_params = clf.best_params_
svm_pipe = make_pipeline(StandardScaler(), SVC(C=best_params['svc__C'], kernel=best_params['svc__kernel'], gamma=best_params['svc__gamma']))
start_time = time.time()
svm_pipe.fit(X_train_VS, y_train)
end_time = time.time()
print(f"Model Fitting Time: {end_time - start_time}")

Model Fitting Time: 0.024572134017944336


In [18]:
y_pred = svm_pipe.predict(X_test_VS)
print(classification_report(y_test, y_pred))
print('Confusion Matrix:\n', confusion_matrix(y_pred,y_test),'\n')
print('Accuracy:', round(accuracy_score(y_pred,y_test)*100,2),'%')

              precision    recall  f1-score   support

           0       1.00      0.98      0.99      2821
           1       0.84      0.97      0.90       340

    accuracy                           0.98      3161
   macro avg       0.92      0.98      0.95      3161
weighted avg       0.98      0.98      0.98      3161

Confusion Matrix:
 [[2759    9]
 [  62  331]] 

Accuracy: 97.75 %


### Bi-directional Elimination Variable Selection

##### 'Bidder_Tendency', 'Bidding_Ratio', 'Successive_Outbidding', 'Last_Bidding', 'Auction_Duration'

In [19]:
#Variable Selection
X_train_VS=X_train[0:,(0,1,2,3,8)]
y_train_VS=y_train
X_test_VS=X_test[0:,(0,1,2,3,8)]
y_test_VS=y_test

#### Best Model Part 2 (SVM LINEAR)

In [20]:
model = LinearSVC(penalty='l2', dual=False, max_iter=10000)
param_grid = {'C': [0.01, 0.1, 1, 10, 100]}

start_time = time.time()
# Set up k-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
# Set up GridSearchCV to perform model selection using k-fold cross-validation
clf = GridSearchCV(model, param_grid=param_grid, cv=kfold)
clf.fit(X_train, y_train)
end_time = time.time()
duration1=end_time-start_time
# Get the best model
best_model = clf.best_estimator_

# Evaluate the best model on the testing data
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred))
print('Confusion Matrix:\n', confusion_matrix(y_pred,y_test),'\n')
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {round(accuracy*100,2)}%")

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      2821
           1       0.83      0.95      0.89       340

    accuracy                           0.97      3161
   macro avg       0.91      0.96      0.94      3161
weighted avg       0.98      0.97      0.98      3161

Confusion Matrix:
 [[2757   17]
 [  64  323]] 

Accuracy: 97.44%


#### Best Model Part 3 (SVM Non-Linear)

In [21]:
# SVM Nonlinear

# Set up the pipeline
svm_pipe = make_pipeline(StandardScaler(), SVC())

# Set up the grid search
param_grid = {
    'svc__C': [0.1, 1, 10],
    'svc__kernel': ['poly', 'rbf', 'sigmoid'],
    'svc__gamma': ['scale', 'auto']
}
clf = GridSearchCV(svm_pipe, param_grid, cv=5)
# Fit the model and time it
clf.fit(X_train_VS, y_train)

# Use the best parameters to fit a new SVM model
best_params = clf.best_params_
svm_pipe = make_pipeline(StandardScaler(), SVC(C=best_params['svc__C'], kernel=best_params['svc__kernel'], gamma=best_params['svc__gamma']))
start_time = time.time()
svm_pipe.fit(X_train_VS, y_train)
end_time = time.time()

In [22]:
y_pred = svm_pipe.predict(X_test_VS)
print(classification_report(y_test, y_pred))
print('Confusion Matrix:\n', confusion_matrix(y_pred,y_test),'\n')
print('Accuracy:', round(accuracy_score(y_pred,y_test)*100,2),'%')

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      2821
           1       0.94      0.98      0.96       340

    accuracy                           0.99      3161
   macro avg       0.97      0.98      0.97      3161
weighted avg       0.99      0.99      0.99      3161

Confusion Matrix:
 [[2798    8]
 [  23  332]] 

Accuracy: 99.02 %


## Conclusion:
### In conclusion, we observed best performance in SVM-non linear from bi-directional feature selection where the achieved accuracy score is 99.02 compared to all other feature selection with best models.